# Tulime network analysis

The data regarding the project was first obtained through an Gsheet compiled partially by an organization component and partially by me, employing the data provided by the association thrugh mails and Whatsapp conversations. <br>
This sheet served, once completed, to extract data regarding single projects and convert them into CSVs in order to later access them through python and perform analysis and operation on data through <a href="https://pandas.pydata.org/">Pandas</a> library.

In [13]:
import numpy as np
import pandas as pd
import random

In [20]:
df = pd.read_csv("Maji.csv", encoding="utf-8")
rand_kitowo=random.randint(2500, 3000)
rand_pomerini=random.randint(850, 900)
rand_mwawambala=random.randint(850, 900)
for n in range(0,rand_kitowo):
    df_temp={"NAME": "Name"+ str(n), "VILLAGE": "Kitowo", "AGE": "", "SEX": "", "FAMILY": "", "START": 2015}
    df_temp=pd.DataFrame.from_dict(df_temp, orient="index")
    df = pd.concat([df, df_temp], ignore_index=True)
for n in range(0, rand_pomerini):
    df_temp={"NAME": "Name"+ str(n), "VILLAGE": "Pomerini", "AGE": "", "SEX": "", "FAMILY": "", "START": 2015}
    pd.concat([df, df_temp])
for n in range(0,rand_mwawambala):
    df_temp={"NAME": "Name"+str(n), "VILLAGE": "Mwawambala", "AGE": "", "SEX": "", "FAMILY": "", "START": 2015}
    pd.concat([df, df_temp])
print(df)

FileNotFoundError: [Errno 2] No such file or directory: 'Maji.csv'

In [2]:
def get_all_in_dir(dir):
    for filename in os.listdir(dir):
        f = os.path.join(dir, filename)
        if os.path.isfile(f) and f[-4:] == ".csv":
            yield f

In this first step we need to obtain the distribution of values absent in other CSVs from the that present a more complete amount of information. To do so we first of all extract the data we have and organize it in dictionaries (to conver in CSVs) containing the values of interest and the numbre of peope in realtion to each of the values.

In [98]:
to_scan= [file for file in get_all_in_dir("projects_csv")]

sex_dict = dict()
age_dict=dict()
age_dict_children = dict()
family_dict=dict()

for file in to_scan:
    df = pd.read_csv(file, encoding="utf-8")
    df=df.fillna("NaN")
    for col in df.columns:
        if col == "SEX":
            for el in df[col]:
                if el in sex_dict.keys():
                    sex_dict[el]+=1
                elif el !="NaN":
                    sex_dict[el]=1

        if col == "AGE" and file not in ["projects_csv\TupoPamoja.csv", "projects_csv\Mwalimu.csv" , "projects_csv\PiantalaSubito.csv"]:
            for el in df[col]:
                if el in age_dict.keys():
                    age_dict[el]+=1
                elif el !="NaN":
                    age_dict[el]=1

        if col== "AGE":
            for el in df[col]:
                if el in age_dict_children.keys():
                    age_dict_children[el]+=1
                elif el !="NaN":
                    age_dict_children[el]=1

        
        if col == "FAMILY":
            for el in df[col]:
                if el in family_dict.keys():
                    family_dict[el]+=1
                elif el !="NaN":
                    family_dict[el]=1

df_sex = pd.DataFrame.from_dict(sex_dict, orient='index')
df_sex.to_csv(r"projects_csv/measures/sex_count.csv")

age_dict = dict(sorted(age_dict.items(), key=lambda item: item[1], reverse=True))
df_age = pd.DataFrame.from_dict(age_dict, orient='index')
df_age.to_csv(r"projects_csv/measures/age_count.csv")

age_dict_children = dict(sorted(age_dict_children.items(), key=lambda item: item[1], reverse=True))
df_age_children = pd.DataFrame.from_dict(age_dict_children, orient='index')
df_age_children.to_csv(r"projects_csv/measures/agechildren_count.csv")

family_dict = dict(sorted(family_dict.items(), key=lambda item: item[1], reverse=True))
df_family = pd.DataFrame.from_dict(family_dict, orient='index')
df_family.to_csv(r"projects_csv/measures/family_count.csv")


            


# Extracting distributions from the values obtained
In order to fill missing gaps in the other CSVs we need to transform the data already in our possess into distributions of variables.

In [180]:
sum_sex = 0
sum_age = 0
sum_agechildren = 0
sum_family = 0

for el in sex_dict.keys():
    sum_sex += sex_dict[el]
for el in sex_dict.keys():
    sex_dict[el]=sex_dict[el]/sum_sex

for el in age_dict.keys():
    sum_age += age_dict[el]
for el in age_dict.keys():
    age_dict[el]=age_dict[el]/sum_age

for el in age_dict_children.keys():
    sum_agechildren += age_dict_children[el]
for el in age_dict_children.keys():
    age_dict_children[el]=age_dict_children[el]/sum_agechildren

for el in family_dict.keys():
    sum_family += family_dict[el]
for el in family_dict.keys():
    family_dict[el]=family_dict[el]/sum_family

df_sex = pd.DataFrame.from_dict(sex_dict, orient='index')
df_sex.to_csv(r"projects_csv/measures/sex_distribution.csv")

age_dict = dict(sorted(age_dict.items(), key=lambda item: item[1], reverse=True))
df_age = pd.DataFrame.from_dict(age_dict, orient='index')
df_age.to_csv(r"projects_csv/measures/age_distribution.csv")

age_dict_children = dict(sorted(age_dict_children.items(), key=lambda item: item[1], reverse=True))
df_age_children = pd.DataFrame.from_dict(age_dict_children, orient='index')
df_age_children.to_csv(r"projects_csv/measures/agetotal_distribution.csv")

family_dict = dict(sorted(family_dict.items(), key=lambda item: item[1], reverse=True))
df_family = pd.DataFrame.from_dict(family_dict, orient='index')
df_family.to_csv(r"projects_csv/measures/family_distribution.csv")


# Filling missing values in CSVs using obtained distributions
Now we need to use those values to fill missing cells in the remaining CSVs. WE will care about exceptions such as TupoPamoja and Mwalimu, that have restricted age range and consider the total age CSV just for MAJI, which is a project devoted to water supply: this could seem an uneveness in balance, since children are high in number but one of te interventions of MAJI regards only the school district, that by itself counts 900 inhabitants of younger ages with respect to other villagers. <br>
In order to do so <a href="https://numpy.org/doc/stable/">Numpy</a> was employed.

In [64]:
to_scan= [file for file in get_all_in_dir("projects_csv")]

df_age = pd.read_csv("projects_csv/measures/age_distribution.csv")
df_age_total = pd.read_csv("projects_csv/measures/agetotal_distribution.csv")
df_sex = pd.read_csv("projects_csv/measures/sex_distribution.csv")
df_family = pd.read_csv("projects_csv/measures/family_distribution.csv")

for file in to_scan:
    df = pd.read_csv(file, encoding="utf-8")
    missing_age = df["AGE"].isnull()
    missing_sex = df["SEX"].isnull()
    missing_family = df["FAMILY"].isnull()
    if file.split("\\")[1] not in ["Maji.csv", "Mwalimu.csv"]:
        df.loc[missing_age, "AGE"] = np.random.choice(df_age["AGE"], size=len(df[missing_age]), p=df_age["PERCENTAGE"])
    elif file.split("\\")[1] not in ["TupoPamoja.csv", "Mwalimu.csv"]:
        df.loc[missing_age, "AGE"] = np.random.choice(df_age_total["AGE"], size=len(df[missing_age]), p=df_age_total["PERCENTAGE"])
    df.loc[missing_sex, "SEX"] = np.random.choice(df_sex["SEX"], size=len(df[missing_sex]), p=df_sex["PERCENTAGE"])
    df.loc[missing_family, "FAMILY"] = np.random.choice(df_family["FAMILY"], size=len(df[missing_family]), p=df_family["PERCENTAGE"])
    if file.split("\\")[1] == "TupoPamoja.csv":
        df.loc[missing_age, "AGE"] = np.random.choice(range(18, 49, 6), size=len(df[missing_age]))
        for row in df.iterrows():
            age = row[1]["AGE"]
            idx = df[df['AGE']==age].index.values.astype(int)[0]
            new_age = float("{:.2f}".format(age/12))
            df.loc[idx, "AGE"] = new_age
    elif file.split("\\")[1] == "Mwalimu.csv":
        df.loc[missing_age, "AGE"] = np.random.choice(range(18, 24), size=len(df[missing_age]))
   
    df.to_csv(r"projects_csv/complete/complete_"+file.split("\\")[1])